# [`S2FFT`](https://github.com/astro-informatics/s2fft) - __Spherical harmonic transform__ Interactive Tutorial
---



This tutorial demonstrates how to call the spherical harmonic transform apis within `S2FFT`. Specifically we will be working with the forward and inverse spin-s spherical harmonic transforms (see [McEwen & Wiaux](https://arxiv.org/pdf/1110.6298.pdf)). First we must load an input signal which is correctly sampled on the sphere, one such image could be that of the Galactic plane captured by ESA's [Gaia satellite](https://sci.esa.int/web/gaia)!

In [ ]:
import numpy as np
import s2fft 

L = 1000
sampling = "mw"
f = np.load('data/Gaia_EDR3_flux.npy')

Now, lets take a look at the data on the sphere using [`PyVista`](https://docs.pyvista.org/index.html). **Hint:** try moving the camera inside to see what you would see from the earth!

In [ ]:
s2fft.utils.plotting_functions.plot_sphere(f, L, sampling, isnotebook=True)

### Computing the spherical harmonic transform
---

Lets now run the JAX function to compute the spherical harmonic transform of this observational map.

In [ ]:
flm = s2fft.forward_jax(f, L, reality=True)

If you are planning on applying this transform many times (e.g. during training of a model) we recommend precomputing and storing some small arrays which are used every time. To do this simply compute these and pass as a static argument

In [ ]:
precomps = s2fft.generate_precomputes_jax(L, forward=True)
flm_pre = s2fft.forward_jax(f, L, reality=True, precomps=precomps)

### Computing the inverse spherical harmonic transform
---

Lets run the JAX function to get us back to the observational map, or at least a bandlimited version of said map.

In [ ]:
f_test = s2fft.inverse_jax(flm, L, reality=True)

Again, if you are planning on applying this transform many times (e.g. during training of a model) we recommend precomputing and storing some small arrays which are used every time. To do this simply compute these and pass as a static argument

In [ ]:
precomps = s2fft.generate_precomputes_jax(L, forward=False)
f_pre = s2fft.inverse_jax(flm_pre, L, reality=True, precomps=precomps)

We can now quickly check the associated error

In [ ]:
print(f"Mean absolute error = {np.nanmean(np.abs(np.real(f_test) - f))}")